# Install Package

In [50]:
!pip install  -q deltalake  --upgrade
!pip install  -q duckdb     --upgrade
!pip install  -q polars     --pre --upgrade
!pip install  -q daft
!pip install     chdb==3.7.0
!pip install  -q pysail
!pip install      grpcio --upgrade
!pip install grpcio-status
!pip install pandas         --upgrade
notebookutils.session.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
sys.exit called with value 0. The interpreter will be restarted.


In [51]:
SF= 1

***Import***

In [52]:
import time
import os
import pandas as pd
from   datetime import datetime
from   psutil import *
import duckdb
import pathlib
from   deltalake import convert_to_deltalake , write_deltalake , DeltaTable ,QueryBuilder

In [53]:
core=cpu_count()
core

2

In [54]:
result_path = '/lakehouse/default/Tables/dbo/result'
Table_Path  = '/lakehouse/default/Tables'
Files_Path  = '/lakehouse/default/Files'

# Generate Data

In [55]:
tbl_list = ['web_site','web_sales','web_returns','web_page','warehouse','time_dim','store_sales','store_returns',\
      'store','ship_mode','reason','promotion','item','inventory','income_band','household_demographics','date_dim','customer_demographics',\
      'customer_address','customer','catalog_sales','catalog_returns','catalog_page','call_center']

In [56]:
%%time
if not pathlib.Path(f'{Table_Path}/CH{SF}').exists():
  if not pathlib.Path(f'{Files_Path}/tpch_{SF}.duckdb').exists():
    print('generate new database')
    pathlib.Path(f'{Files_Path}').mkdir(parents=True, exist_ok=True)
    con=duckdb.connect(f'{Files_Path}/tpch_{SF}.duckdb')
    con.sql(f"CALL dsdgen(sf={SF}) ")
  else:
     con=duckdb.connect(f'{Files_Path}/tpch_{SF}.duckdb',read_only=1)
     print('DB exist')
  for tbl in tbl_list :
      pathlib.Path(f'{Table_Path}/CH{SF}/{tbl}').mkdir(parents=True, exist_ok=True)
      con.sql(f"""  COPY (SELECT * FROM {tbl}) TO '{Table_Path}/CH{SF}/{tbl}' (FORMAT PARQUET,PER_THREAD_OUTPUT TRUE)  """)
      convert_to_deltalake(f'{Table_Path}/CH{SF}/{tbl}',storage_options={"allow_unsafe_rename":"true"})
  con.close()
else:
  print("Already Generated")

Already Generated
CPU times: user 1.15 ms, sys: 0 ns, total: 1.15 ms
Wall time: 1.46 ms


# ------------------

# SQL Query

In [57]:
# @title
sql = """
-- SQL Engine Coverage Test Suite - Advanced SQL Pattern Testing
-- Comprehensive test of SQL syntax coverage across different database engines
-- Total: 44 SQL Queries (41 standard + 3 engine-specific limitation tests)

-- ==========================================
-- SECTION 1: BASIC SQL FUNDAMENTALS
-- ==========================================

-- Query 1: Basic SELECT with simple filtering
SELECT i_item_id, i_item_desc, i_current_price
FROM item
WHERE i_current_price > 100
AND i_category = 'Electronics'
ORDER BY i_current_price DESC
LIMIT 10;

-- Query 2: Basic JOIN (most common pattern)
SELECT c.c_customer_id, c.c_first_name, s.ss_sales_price
FROM customer c
INNER JOIN store_sales s ON c.c_customer_sk = s.ss_customer_sk
WHERE s.ss_sales_price > 50;

-- Query 3: Basic GROUP BY with HAVING
SELECT i_category, count(*) AS item_count, avg(i_current_price) AS avg_price
FROM item
GROUP BY i_category
HAVING count(*) > 100
ORDER BY avg_price DESC;

-- Query 4: Basic subquery
SELECT c_customer_id
FROM customer
WHERE c_customer_sk IN (
    SELECT ss_customer_sk
    FROM store_sales
    WHERE ss_sales_price > 100
);

-- Query 5: Basic UNION
SELECT 'Store' AS channel, count(*) AS sales_count
FROM store_sales
UNION ALL
SELECT 'Web' AS channel, count(*) AS sales_count
FROM web_sales;

-- ==========================================
-- SECTION 3: QUALIFY CLAUSE PATTERNS
-- ==========================================

-- Query 6: RANK with subquery - Top N per category (Standard SQL)
SELECT
    i_category,
    i_item_id,
    i_current_price,
    price_rank
FROM (
    SELECT
        i_category,
        i_item_id,
        i_current_price,
        RANK() OVER (PARTITION BY i_category ORDER BY i_current_price DESC) as price_rank
    FROM item
) ranked
WHERE price_rank <= 3;


-- Query 7: QUALIFY with ROW_NUMBER() - Get latest record per group
SELECT
    c_customer_id,
    c_first_name,
    c_last_name,
    ss_sold_date_sk,
    ss_sales_price
FROM customer
INNER JOIN store_sales ON c_customer_sk = ss_customer_sk
INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
QUALIFY ROW_NUMBER() OVER (PARTITION BY c_customer_sk ORDER BY d_date DESC) = 1;



-- Query 8: DENSE_RANK with subquery - Handle ties differently (Standard SQL)
SELECT
    s_store_name,
    s_state,
    total_sales,
    sales_rank
FROM (
    SELECT
        s_store_name,
        s_state,
        total_sales,
        DENSE_RANK() OVER (ORDER BY total_sales DESC) as sales_rank
    FROM (
        SELECT s_store_name, s_state, sum(ss_ext_sales_price) as total_sales
        FROM store
        INNER JOIN store_sales ON s_store_sk = ss_store_sk
        GROUP BY s_store_name, s_state
    ) store_totals
) ranked_stores
WHERE sales_rank <= 5;

-- Query 9: LAG/LEAD with subquery - Compare with previous/next values (Standard SQL)
SELECT
    d_date,
    daily_sales,
    prev_day_sales,
    next_day_sales
FROM (
    SELECT
        d_date,
        daily_sales,
        LAG(daily_sales) OVER (ORDER BY d_date) as prev_day_sales,
        LEAD(daily_sales) OVER (ORDER BY d_date) as next_day_sales
    FROM (
        SELECT d_date, sum(ss_ext_sales_price) as daily_sales
        FROM date_dim
        INNER JOIN store_sales ON d_date_sk = ss_sold_date_sk
        WHERE d_year = 2001
        GROUP BY d_date
    ) daily_totals
) with_lag_lead
WHERE prev_day_sales IS NOT NULL AND next_day_sales IS NOT NULL;

-- Query 10: Percentile functions with subquery (Standard SQL)
SELECT
    i_item_id,
    i_current_price,
    price_percentile
FROM (
    SELECT
        i_item_id,
        i_current_price,
        PERCENT_RANK() OVER (ORDER BY i_current_price) as price_percentile
    FROM item
    WHERE i_category = 'Electronics'
) ranked
WHERE price_percentile >= 0.95;  -- Top 5% by price

-- ==========================================
-- SECTION 4: PATTERN MATCHING WITH WINDOW FUNCTIONS (MATCH_RECOGNIZE Alternative)
-- ==========================================

-- Query 11: Customer purchase sequence analysis using window functions (DuckDB compatible)
WITH customer_purchases AS (
    SELECT
        c_customer_id,
        d_date,
        ss_ext_sales_price,
        LAG(ss_ext_sales_price) OVER (PARTITION BY c_customer_id ORDER BY d_date) as prev_spend,
        ROW_NUMBER() OVER (PARTITION BY c_customer_id ORDER BY d_date) as purchase_seq
    FROM customer
    INNER JOIN store_sales ON c_customer_sk = ss_customer_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_year = 2001
),
increasing_flags AS (
    SELECT
        *,
        CASE WHEN ss_ext_sales_price > prev_spend THEN 1 ELSE 0 END as is_increasing,
        -- Create groups for consecutive increasing sequences
        SUM(CASE WHEN ss_ext_sales_price <= COALESCE(prev_spend, 0) THEN 1 ELSE 0 END)
            OVER (PARTITION BY c_customer_id ORDER BY d_date) as grp_id
    FROM customer_purchases
    WHERE prev_spend IS NOT NULL
),
increasing_sequences AS (
    SELECT
        c_customer_id,
        grp_id,
        MIN(d_date) as pattern_start,
        MAX(d_date) as pattern_end,
        COUNT(*) as purchase_count
    FROM increasing_flags
    WHERE is_increasing = 1
    GROUP BY c_customer_id, grp_id
    HAVING COUNT(*) >= 2  -- At least 2 consecutive increases
)
SELECT * FROM increasing_sequences
ORDER BY c_customer_id, pattern_start;

-- ==========================================
-- SECTION 5: FILTER WHERE CLAUSE PATTERNS
-- ==========================================

-- Query 12: FILTER WHERE with window functions - Calculate conditional aggregates
SELECT
    i_category,
    i_class,
    count(*) as total_items,
    count(*) FILTER (WHERE i_current_price > 100) as expensive_items,
    avg(i_current_price) as avg_price,
    avg(i_current_price) FILTER (WHERE i_current_price > 100) as avg_expensive_price,
    -- Use subquery for window function to avoid GROUP BY conflict
    (SELECT count(*) FROM item i2 WHERE i2.i_category = item.i_category AND i2.i_current_price > 100) as expensive_in_category
FROM item
GROUP BY i_category, i_class
ORDER BY i_category, i_class;

-- Query 13: FILTER WHERE with multiple conditions
SELECT
    s_store_name,
    s_state,
    count(*) FILTER (WHERE ss_sales_price > 100) as high_value_sales,
    count(*) FILTER (WHERE ss_sales_price BETWEEN 50 AND 100) as medium_value_sales,
    count(*) FILTER (WHERE ss_sales_price < 50) as low_value_sales,
    sum(ss_sales_price) FILTER (WHERE d_day_name = 'Saturday') as weekend_sales,
    avg(ss_sales_price) FILTER (WHERE cd_marital_status = 'M') as married_customer_avg
FROM store
INNER JOIN store_sales ON s_store_sk = ss_store_sk
INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
INNER JOIN customer_demographics ON ss_cdemo_sk = cd_demo_sk
WHERE d_year = 2001
GROUP BY s_store_name, s_state;

-- ==========================================
-- SECTION 6: ADVANCED SQL FEATURES
-- ==========================================

-- Query 14: PIVOT operation (using CASE WHEN for databases without native PIVOT)
SELECT
    i_category,
    sum(CASE WHEN d_qoy = 1 THEN ss_ext_sales_price ELSE 0 END) as Q1_sales,
    sum(CASE WHEN d_qoy = 2 THEN ss_ext_sales_price ELSE 0 END) as Q2_sales,
    sum(CASE WHEN d_qoy = 3 THEN ss_ext_sales_price ELSE 0 END) as Q3_sales,
    sum(CASE WHEN d_qoy = 4 THEN ss_ext_sales_price ELSE 0 END) as Q4_sales
FROM item
INNER JOIN store_sales ON i_item_sk = ss_item_sk
INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
WHERE d_year = 2001
GROUP BY i_category;

-- Query 15: UNPIVOT operation simulation
WITH pivoted_data AS (
    SELECT i_category, 'Q1' as quarter, sum(CASE WHEN d_qoy = 1 THEN ss_ext_sales_price ELSE 0 END) as sales
    FROM item
    INNER JOIN store_sales ON i_item_sk = ss_item_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_year = 2001
    GROUP BY i_category
    UNION ALL
    SELECT i_category, 'Q2' as quarter, sum(CASE WHEN d_qoy = 2 THEN ss_ext_sales_price ELSE 0 END) as sales
    FROM item
    INNER JOIN store_sales ON i_item_sk = ss_item_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_year = 2001
    GROUP BY i_category
    UNION ALL
    SELECT i_category, 'Q3' as quarter, sum(CASE WHEN d_qoy = 3 THEN ss_ext_sales_price ELSE 0 END) as sales
    FROM item
    INNER JOIN store_sales ON i_item_sk = ss_item_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_year = 2001
    GROUP BY i_category
    UNION ALL
    SELECT i_category, 'Q4' as quarter, sum(CASE WHEN d_qoy = 4 THEN ss_ext_sales_price ELSE 0 END) as sales
    FROM item
    INNER JOIN store_sales ON i_item_sk = ss_item_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_year = 2001
    GROUP BY i_category
)
SELECT * FROM pivoted_data WHERE sales > 0;

-- Query 16: LATERAL JOIN (for databases that support it) - Fixed with proper correlation
SELECT
    c.c_customer_id,
    c.c_first_name,
    recent_purchases.purchase_count,
    recent_purchases.total_spent
FROM customer c
INNER JOIN LATERAL (
    SELECT
        count(*) as purchase_count,
        sum(ss_ext_sales_price) as total_spent
    FROM store_sales ss
    INNER JOIN date_dim d ON ss.ss_sold_date_sk = d.d_date_sk
    WHERE ss.ss_customer_sk = c.c_customer_sk
    AND d.d_date >= '2001-12-01'::date
) recent_purchases ON true
WHERE recent_purchases.purchase_count > 0;

-- Query 17: TABLESAMPLE for sampling large datasets - Fixed syntax
SELECT c_customer_id, c_first_name, c_last_name
FROM customer TABLESAMPLE SYSTEM (1 PERCENT)
WHERE c_preferred_cust_flag = 'Y';

-- ==========================================
-- SECTION 7: RECURSIVE AND HIERARCHICAL QUERIES
-- ==========================================

-- Query 18: Recursive CTE - Build date hierarchy (Fixed)
WITH RECURSIVE date_hierarchy AS (
    -- Base case: start with a specific date
    SELECT
        d_date_sk,
        d_date,
        d_week_seq,
        d_month_seq,
        d_year,
        0 as level
    FROM date_dim
    WHERE d_date = '2001-01-01'

    UNION ALL

    -- Recursive case: find next dates in sequence
    SELECT
        d.d_date_sk,
        d.d_date,
        d.d_week_seq,
        d.d_month_seq,
        d.d_year,
        dh.level + 1
    FROM date_dim d
    INNER JOIN date_hierarchy dh ON d.d_date_sk = dh.d_date_sk + 1
    WHERE dh.level < 30  -- Limit recursion depth to 30 days
)
SELECT * FROM date_hierarchy
ORDER BY d_date;

-- Query 19: Hierarchical query with proper customer relationships (Fixed)
WITH RECURSIVE customer_address_hierarchy AS (
    SELECT
        c_customer_sk,
        c_customer_id,
        c_first_name,
        c_last_name,
        c_current_addr_sk,
        0 as hierarchy_level,
        ARRAY[c_customer_sk] as path
    FROM customer
    WHERE c_preferred_cust_flag = 'Y'  -- Root level customers

    UNION ALL

    SELECT
        c.c_customer_sk,
        c.c_customer_id,
        c.c_first_name,
        c.c_last_name,
        c.c_current_addr_sk,
        cah.hierarchy_level + 1,
        cah.path || ARRAY[c.c_customer_sk]  -- Cast scalar to array for concatenation
    FROM customer c
    INNER JOIN customer_address_hierarchy cah ON c.c_current_addr_sk = cah.c_current_addr_sk  -- Same address hierarchy
    WHERE cah.hierarchy_level < 2
    AND NOT (c.c_customer_sk = ANY(cah.path))  -- Prevent cycles
    AND c.c_customer_sk != cah.c_customer_sk  -- Prevent self-joins
)
SELECT * FROM customer_address_hierarchy
ORDER BY hierarchy_level, c_customer_id;

-- ==========================================
-- SECTION 8: WINDOW FUNCTION ADVANCED PATTERNS
-- ==========================================

-- Query 20: RANGE vs ROWS window frames
SELECT
    d_date,
    daily_sales,
    -- Rolling 7-day sum using ROWS
    sum(daily_sales) OVER (
        ORDER BY d_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as rolling_7day_rows,
    -- Range-based window (by date values) - using days instead of interval
    sum(daily_sales) OVER (
        ORDER BY d_date
        RANGE BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as rolling_7day_range
FROM (
    SELECT d_date, sum(ss_ext_sales_price) as daily_sales
    FROM date_dim
    INNER JOIN store_sales ON d_date_sk = ss_sold_date_sk
    WHERE d_year = 2001 AND d_moy = 12
    GROUP BY d_date
) daily_totals
ORDER BY d_date;

-- Query 21: Advanced window functions with multiple partitions
SELECT
    i_category,
    i_class,
    i_item_id,
    i_current_price,
    -- Multiple ranking functions
    ROW_NUMBER() OVER (PARTITION BY i_category ORDER BY i_current_price DESC) as price_row_num,
    RANK() OVER (PARTITION BY i_category ORDER BY i_current_price DESC) as price_rank,
    DENSE_RANK() OVER (PARTITION BY i_category ORDER BY i_current_price DESC) as price_dense_rank,
    -- Percentile functions
    PERCENT_RANK() OVER (PARTITION BY i_category ORDER BY i_current_price) as price_percent_rank,
    CUME_DIST() OVER (PARTITION BY i_category ORDER BY i_current_price) as price_cumulative_dist,
    -- NTILE for bucketing
    NTILE(4) OVER (PARTITION BY i_category ORDER BY i_current_price) as price_quartile,
    -- First/Last value functions
    FIRST_VALUE(i_current_price) OVER (
        PARTITION BY i_category
        ORDER BY i_current_price DESC
        ROWS UNBOUNDED PRECEDING
    ) as highest_price_in_category,
    LAST_VALUE(i_current_price) OVER (
        PARTITION BY i_category
        ORDER BY i_current_price DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as lowest_price_in_category,
    -- LAG/LEAD with offsets
    LAG(i_current_price, 2) OVER (PARTITION BY i_category ORDER BY i_current_price) as price_lag_2,
    LEAD(i_current_price, 1, 0) OVER (PARTITION BY i_category ORDER BY i_current_price) as price_lead_with_default
FROM item
WHERE i_category IN ('Electronics', 'Clothing', 'Home')
ORDER BY i_category, i_current_price DESC;

-- Query 22: Pattern 1 - Basic INNER JOIN with Aggregation and Subquery
WITH customer_total_return AS (
    SELECT
        sr_customer_sk AS ctr_customer_sk,
        sr_store_sk AS ctr_store_sk,
        sum(sr_return_amt) AS ctr_total_return
    FROM store_returns
    INNER JOIN date_dim ON sr_returned_date_sk = d_date_sk
    WHERE d_year = 2000
    GROUP BY sr_customer_sk, sr_store_sk
)
SELECT c_customer_id
FROM customer_total_return ctr1
INNER JOIN store ON ctr1.ctr_store_sk = s_store_sk
INNER JOIN customer ON ctr1.ctr_customer_sk = c_customer_sk
WHERE ctr1.ctr_total_return > (
    SELECT avg(ctr_total_return)*1.2
    FROM customer_total_return ctr2
    WHERE ctr1.ctr_store_sk = ctr2.ctr_store_sk
)
AND s_state = 'TN'
ORDER BY c_customer_id
LIMIT 100;

-- Query 23: Pattern 2 - UNION ALL with Window Functions and Complex CTE
WITH wscs AS (
    SELECT sold_date_sk, sales_price
    FROM (
        SELECT ws_sold_date_sk AS sold_date_sk, ws_ext_sales_price AS sales_price FROM web_sales
        UNION ALL
        SELECT cs_sold_date_sk AS sold_date_sk, cs_ext_sales_price AS sales_price FROM catalog_sales
    ) sq1
),
wswscs AS (
    SELECT
        d_week_seq,
        sum(CASE WHEN d_day_name='Sunday' THEN sales_price ELSE NULL END) AS sun_sales,
        sum(CASE WHEN d_day_name='Monday' THEN sales_price ELSE NULL END) AS mon_sales
        -- Additional day aggregations can be added here
    FROM wscs
    INNER JOIN date_dim ON d_date_sk = sold_date_sk
    GROUP BY d_week_seq
)
SELECT
    d_week_seq1,
    round(sun_sales1/NULLIF(sun_sales2, 0), 2) AS ratio
FROM (
    SELECT wswscs.d_week_seq AS d_week_seq1, sun_sales AS sun_sales1
    FROM wswscs
    INNER JOIN date_dim ON date_dim.d_week_seq = wswscs.d_week_seq
    WHERE d_year = 2001
) y
INNER JOIN (
    SELECT wswscs.d_week_seq AS d_week_seq2, sun_sales AS sun_sales2
    FROM wswscs
    INNER JOIN date_dim ON date_dim.d_week_seq = wswscs.d_week_seq
    WHERE d_year = 2002
) z ON d_week_seq1 = d_week_seq2-53
ORDER BY d_week_seq1;

-- Query 24: Pattern 3 - Multiple JOINs with EXISTS and Complex Filtering
SELECT
    cd_gender, cd_marital_status, cd_education_status,
    count(*) AS cnt1, cd_purchase_estimate, count(*) AS cnt2
FROM customer c
INNER JOIN customer_address ca ON c.c_current_addr_sk = ca.ca_address_sk
INNER JOIN customer_demographics ON cd_demo_sk = c.c_current_cdemo_sk
WHERE ca_county IN ('Rush County', 'Toole County', 'Jefferson County')
AND EXISTS (
    SELECT 1
    FROM store_sales
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE c.c_customer_sk = ss_customer_sk
    AND d_year = 2002 AND d_moy BETWEEN 1 AND 4
)
AND (EXISTS (
    SELECT 1 FROM web_sales
    INNER JOIN date_dim ON ws_sold_date_sk = d_date_sk
    WHERE c.c_customer_sk = ws_bill_customer_sk AND d_year = 2002
) OR EXISTS (
    SELECT 1 FROM catalog_sales
    INNER JOIN date_dim ON cs_sold_date_sk = d_date_sk
    WHERE c.c_customer_sk = cs_ship_customer_sk AND d_year = 2002
))
GROUP BY cd_gender, cd_marital_status, cd_education_status, cd_purchase_estimate
ORDER BY cd_gender, cd_marital_status, cd_education_status
LIMIT 100;

-- Query 25: Pattern 4 - LEFT OUTER JOIN with COALESCE and Complex CASE Statements (Fixed)
WITH ssr AS (
    SELECT
        s_store_id,
        sum(sales_price) AS sales,
        sum(return_amt) AS returns_,
        sum(net_loss) AS profit_loss
    FROM (
        SELECT
            ss_store_sk AS store_sk, ss_sold_date_sk AS date_sk,
            ss_ext_sales_price AS sales_price,
            0::decimal(7,2) AS return_amt,
            0::decimal(7,2) AS net_loss
        FROM store_sales
        UNION ALL
        SELECT
            sr_store_sk AS store_sk, sr_returned_date_sk AS date_sk,
            0::decimal(7,2) AS sales_price,
            sr_return_amt AS return_amt,
            sr_net_loss AS net_loss
        FROM store_returns
    ) salesreturns
    INNER JOIN date_dim ON date_sk = d_date_sk
    INNER JOIN store ON store_sk = s_store_sk
    WHERE d_date BETWEEN '2000-08-23'::date AND '2000-09-06'::date
    GROUP BY s_store_id
),
wsr AS (
    SELECT
        'web_site1' AS web_site_id,
        sum(ws_ext_sales_price) AS sales,
        sum(COALESCE(wr_return_amt, 0)) AS returns_,
        sum(ws_ext_sales_price) - sum(COALESCE(wr_return_amt, 0)) AS profit
    FROM web_sales
    LEFT JOIN web_returns ON ws_order_number = wr_order_number
    INNER JOIN date_dim ON ws_sold_date_sk = d_date_sk
    WHERE d_date BETWEEN '2000-08-23'::date AND '2000-09-06'::date
)
SELECT
    'store channel' AS channel,
    CONCAT('store', s_store_id) AS id,
    sales,
    returns_,
    (sales - returns_ - profit_loss) AS profit
FROM ssr
UNION ALL
SELECT
    'web channel' AS channel,
    CONCAT('web_site', web_site_id) AS id,
    sales,
    COALESCE(returns_, 0) AS returns_,
    profit
FROM wsr
ORDER BY channel, id
LIMIT 100;

-- Query 26: Pattern 5 - FULL OUTER JOIN with Window Functions
WITH web_v1 AS (
    SELECT
        ws_item_sk AS item_sk, d_date,
        sum(sum(ws_sales_price)) OVER (
            PARTITION BY ws_item_sk
            ORDER BY d_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cume_sales
    FROM web_sales
    INNER JOIN date_dim ON ws_sold_date_sk = d_date_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211 AND ws_item_sk IS NOT NULL
    GROUP BY ws_item_sk, d_date
),
store_v1 AS (
    SELECT
        ss_item_sk AS item_sk, d_date,
        sum(sum(ss_sales_price)) OVER (
            PARTITION BY ss_item_sk
            ORDER BY d_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cume_sales
    FROM store_sales
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211 AND ss_item_sk IS NOT NULL
    GROUP BY ss_item_sk, d_date
)
SELECT item_sk, d_date, web_sales, store_sales
FROM (
    SELECT
        COALESCE(web.item_sk, store.item_sk) AS item_sk,
        COALESCE(web.d_date, store.d_date) AS d_date,
        web.cume_sales AS web_sales,
        store.cume_sales AS store_sales
    FROM web_v1 web
    FULL OUTER JOIN store_v1 store ON (web.item_sk = store.item_sk AND web.d_date = store.d_date)
) x
ORDER BY item_sk, d_date
LIMIT 100;

-- Query 27: Pattern 6 - Self-JOIN with Complex ON Conditions
WITH inv AS (
    SELECT
        w_warehouse_name, w_warehouse_sk, i_item_sk, d_moy,
        stddev_samp(inv_quantity_on_hand)*1.000 AS stdev,
        avg(inv_quantity_on_hand) AS mean,
        CASE WHEN avg(inv_quantity_on_hand) = 0 THEN NULL
             ELSE stddev_samp(inv_quantity_on_hand)/avg(inv_quantity_on_hand) END AS cov
    FROM inventory
    INNER JOIN item ON inv_item_sk = i_item_sk
    INNER JOIN warehouse ON inv_warehouse_sk = w_warehouse_sk
    INNER JOIN date_dim ON inv_date_sk = d_date_sk
    WHERE d_year = 2001
    GROUP BY w_warehouse_name, w_warehouse_sk, i_item_sk, d_moy
    HAVING CASE WHEN avg(inv_quantity_on_hand) = 0 THEN 0
                ELSE stddev_samp(inv_quantity_on_hand)/avg(inv_quantity_on_hand) END > 1
)
SELECT
    inv1.w_warehouse_sk, inv1.i_item_sk, inv1.d_moy AS dmoy1,
    inv2.d_moy AS dmoy2, inv1.mean, inv2.mean
FROM inv inv1
INNER JOIN inv inv2 ON inv1.i_item_sk = inv2.i_item_sk
                    AND inv1.w_warehouse_sk = inv2.w_warehouse_sk
                    AND inv1.d_moy = 1 AND inv2.d_moy = 2
ORDER BY inv1.w_warehouse_sk;

-- Query 28: Pattern 7 - INTERSECT with Multiple Date Conditions
SELECT count(*)
FROM (
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM store_sales
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    INNER JOIN customer ON ss_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211
    INTERSECT
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM catalog_sales
    INNER JOIN date_dim ON cs_sold_date_sk = d_date_sk
    INNER JOIN customer ON cs_bill_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211
    INTERSECT
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM web_sales
    INNER JOIN date_dim ON ws_sold_date_sk = d_date_sk
    INNER JOIN customer ON ws_bill_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211
) hot_cust
LIMIT 100;

-- Query 29: Pattern 8 - EXCEPT with Multiple Conditions
SELECT count(*)
FROM ((
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM store_sales
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    INNER JOIN customer ON ss_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211
) EXCEPT (
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM catalog_sales
    INNER JOIN date_dim ON cs_sold_date_sk = d_date_sk
    INNER JOIN customer ON cs_bill_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211
) EXCEPT (
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM web_sales
    INNER JOIN date_dim ON ws_sold_date_sk = d_date_sk
    INNER JOIN customer ON ws_bill_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1200 AND 1211
)) cool_cust;

-- Query 30: Pattern 9 - ROLLUP with Complex Grouping and Ranking
WITH results AS (
    SELECT
        sum(ss_net_profit) AS ss_net_profit,
        sum(ss_ext_sales_price) AS ss_ext_sales_price,
        CASE WHEN sum(ss_ext_sales_price) > 0
             THEN (sum(ss_net_profit)*1.0000)/sum(ss_ext_sales_price)
             ELSE 0 END AS gross_margin,
        i_category, i_class
    FROM store_sales
    INNER JOIN date_dim d1 ON ss_sold_date_sk = d1.d_date_sk
    INNER JOIN item ON ss_item_sk = i_item_sk
    INNER JOIN store ON ss_store_sk = s_store_sk
    WHERE d1.d_year = 2001 AND s_state = 'TN'
    GROUP BY i_category, i_class
)
SELECT
    gross_margin, i_category, i_class, lochierarchy,
    rank() OVER (
        PARTITION BY lochierarchy, CASE WHEN t_class = 0 THEN i_category END
        ORDER BY gross_margin ASC
    ) AS rank_within_parent
FROM (
    SELECT gross_margin, i_category, i_class, 0 AS lochierarchy, 0 AS t_class FROM results
    UNION
    SELECT
        CASE WHEN sum(ss_ext_sales_price) > 0
             THEN (sum(ss_net_profit)*1.0000)/sum(ss_ext_sales_price)
             ELSE 0 END AS gross_margin,
        i_category, NULL AS i_class, 1 AS lochierarchy, 1 AS t_class
    FROM results GROUP BY i_category
) results_rollup
ORDER BY lochierarchy DESC, rank_within_parent
LIMIT 100;

-- Query 31: Pattern 10 - Complex Subquery with NOT EXISTS
SELECT
    i_item_id, i_item_desc, i_current_price
FROM item
INNER JOIN inventory ON inv_item_sk = i_item_sk
INNER JOIN date_dim ON d_date_sk = inv_date_sk
INNER JOIN catalog_sales ON cs_item_sk = i_item_sk
WHERE i_current_price BETWEEN 68 AND 98
AND d_date BETWEEN '2000-02-01'::date AND '2000-04-01'::date
AND i_manufact_id IN (677,940,694,808)
AND inv_quantity_on_hand BETWEEN 100 AND 500
AND NOT EXISTS (
    SELECT 1
    FROM catalog_returns cr1
    WHERE cs_order_number = cr1.cr_order_number
)
GROUP BY i_item_id, i_item_desc, i_current_price
ORDER BY i_item_id
LIMIT 100;

-- Query 32: Pattern 11 - Complex CASE Expressions with Aggregation
SELECT
    CASE
        WHEN (SELECT count(*) FROM store_sales WHERE ss_quantity BETWEEN 1 AND 20) > 74129
        THEN (SELECT avg(ss_ext_discount_amt) FROM store_sales WHERE ss_quantity BETWEEN 1 AND 20)
        ELSE (SELECT avg(ss_net_paid) FROM store_sales WHERE ss_quantity BETWEEN 1 AND 20)
    END AS bucket1,
    CASE
        WHEN (SELECT count(*) FROM store_sales WHERE ss_quantity BETWEEN 21 AND 40) > 122840
        THEN (SELECT avg(ss_ext_discount_amt) FROM store_sales WHERE ss_quantity BETWEEN 21 AND 40)
        ELSE (SELECT avg(ss_net_paid) FROM store_sales WHERE ss_quantity BETWEEN 21 AND 40)
    END AS bucket2
FROM reason
WHERE r_reason_sk = 1;

-- Query 33: Pattern 12 - Multiple CTEs with Different JOIN Types (FIXED)
WITH cross_items AS (
    SELECT i_item_sk AS ss_item_sk
    FROM item
    INNER JOIN (
        SELECT iss.i_brand_id AS brand_id, iss.i_class_id AS class_id, iss.i_category_id AS category_id
        FROM store_sales
        INNER JOIN item iss ON ss_item_sk = iss.i_item_sk
        INNER JOIN date_dim d1 ON ss_sold_date_sk = d1.d_date_sk
        WHERE d1.d_year BETWEEN 1999 AND 2001
        INTERSECT
        SELECT ics.i_brand_id, ics.i_class_id, ics.i_category_id
        FROM catalog_sales
        INNER JOIN item ics ON cs_item_sk = ics.i_item_sk
        INNER JOIN date_dim d2 ON cs_sold_date_sk = d2.d_date_sk
        WHERE d2.d_year BETWEEN 1999 AND 2001
    ) sq1 ON i_brand_id = brand_id AND i_class_id = class_id AND i_category_id = category_id
),
avg_sales AS (
    SELECT avg(quantity*list_price) AS average_sales
    FROM (
        SELECT ss_quantity AS quantity, ss_list_price AS list_price
        FROM store_sales
        INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
        WHERE d_year BETWEEN 1999 AND 2001
        UNION ALL
        SELECT cs_quantity AS quantity, cs_list_price AS list_price
        FROM catalog_sales
        INNER JOIN date_dim ON cs_sold_date_sk = d_date_sk
        WHERE d_year BETWEEN 1999 AND 2001
    ) sq2
)
SELECT channel, i_brand_id, sum(sales) AS sum_sales
FROM (
    SELECT 'store' AS channel, i_brand_id, sum(ss_quantity*ss_list_price) AS sales
    FROM store_sales
    INNER JOIN item ON ss_item_sk = i_item_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE ss_item_sk IN (SELECT ss_item_sk FROM cross_items)
    AND d_year = 2001 AND d_moy = 11
    GROUP BY i_brand_id
    HAVING sum(ss_quantity*ss_list_price) > (SELECT average_sales FROM avg_sales)
) y
GROUP BY ROLLUP (channel, i_brand_id)
ORDER BY channel, i_brand_id
LIMIT 100;

-- Query 34: Pattern 13 - Warehouse Time Analysis with Multiple Joins
SELECT
    w_warehouse_name, w_warehouse_sq_ft, w_city, w_state, ship_carriers, year_,
    sum(jan_sales) AS jan_sales, sum(feb_sales) AS feb_sales,
    sum(jan_sales/NULLIF(w_warehouse_sq_ft, 0)) AS jan_sales_per_sq_foot
FROM (
    SELECT
        w_warehouse_name, w_warehouse_sq_ft, w_city, w_state,
        'DHL,BARIAN' AS ship_carriers, d_year AS year_,
        sum(case when d_moy = 1 then ws_ext_sales_price* ws_quantity else 0 end) AS jan_sales,
        sum(case when d_moy = 2 then ws_ext_sales_price* ws_quantity else 0 end) AS feb_sales
    FROM web_sales
    INNER JOIN warehouse ON ws_warehouse_sk = w_warehouse_sk
    INNER JOIN date_dim ON ws_sold_date_sk = d_date_sk
    INNER JOIN time_dim ON ws_sold_time_sk = t_time_sk
    INNER JOIN ship_mode ON ws_ship_mode_sk = sm_ship_mode_sk
    WHERE d_year = 2001
    AND t_time BETWEEN 30838 AND 30838+28800
    AND sm_carrier IN ('DHL','BARIAN')
    GROUP BY w_warehouse_name, w_warehouse_sq_ft, w_city, w_state, d_year
) x
GROUP BY w_warehouse_name, w_warehouse_sq_ft, w_city, w_state, ship_carriers, year_
ORDER BY w_warehouse_name
LIMIT 100;

-- Query 35: Pattern 14 - Rank and Dense_Rank with Complex Partitioning
WITH best_performing AS (
    SELECT
        item_sk,
        rnk
    FROM (
        SELECT
            item_sk,
            rank() OVER (ORDER BY rank_col ASC) AS rnk
        FROM (
            SELECT ss_item_sk AS item_sk, avg(ss_net_profit) AS rank_col
            FROM store_sales ss1
            WHERE ss_store_sk = 4
            GROUP BY ss_item_sk
            HAVING avg(ss_net_profit) > 0.9*COALESCE((
                SELECT avg(ss_net_profit)
                FROM store_sales
                WHERE ss_store_sk = 4 AND ss_addr_sk IS NULL
                GROUP BY ss_store_sk
            ), 0)
        ) V1
    ) ranked
    WHERE rnk < 11
),
worst_performing AS (
    SELECT
        item_sk,
        rnk
    FROM (
        SELECT
            item_sk,
            rank() OVER (ORDER BY rank_col DESC) AS rnk
        FROM (
            SELECT ss_item_sk AS item_sk, avg(ss_net_profit) AS rank_col
            FROM store_sales ss1
            WHERE ss_store_sk = 4
            GROUP BY ss_item_sk
            HAVING avg(ss_net_profit) > 0.9*COALESCE((
                SELECT avg(ss_net_profit)
                FROM store_sales
                WHERE ss_store_sk = 4 AND ss_addr_sk IS NULL
                GROUP BY ss_store_sk
            ), 0)
        ) V2
    ) ranked
    WHERE rnk < 11
)
SELECT
    bp.rnk,
    i1.i_product_name AS best_performing,
    i2.i_product_name AS worst_performing
FROM best_performing bp
INNER JOIN worst_performing wp ON bp.rnk = wp.rnk
INNER JOIN item i1 ON i1.i_item_sk = bp.item_sk
INNER JOIN item i2 ON i2.i_item_sk = wp.item_sk
ORDER BY bp.rnk
LIMIT 100;

-- Query 36: Pattern 15 - LEFT JOIN with Ratio Calculations (Fixed column references)
WITH ws AS (
    SELECT
        d_year AS ws_sold_year, ws_item_sk, ws_bill_customer_sk AS ws_customer_sk,
        sum(ws_quantity) AS ws_qty, sum(ws_wholesale_cost) AS ws_wc, sum(ws_sales_price) AS ws_sp
    FROM web_sales
    LEFT JOIN web_returns ON wr_order_number = ws_order_number AND ws_item_sk = wr_item_sk
    INNER JOIN date_dim ON ws_sold_date_sk = d_date_sk
    WHERE wr_order_number IS NULL
    GROUP BY d_year, ws_item_sk, ws_bill_customer_sk
),
ss AS (
    SELECT
        d_year AS ss_sold_year, ss_item_sk, ss_customer_sk,
        sum(ss_quantity) AS ss_qty, sum(ss_wholesale_cost) AS ss_wc, sum(ss_sales_price) AS ss_sp
    FROM store_sales
    LEFT JOIN store_returns ON sr_ticket_number = ss_ticket_number AND ss_item_sk = sr_item_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE sr_ticket_number IS NULL
    GROUP BY d_year, ss_item_sk, ss_customer_sk
)
SELECT
    ss_sold_year, ss_item_sk, ss_customer_sk,
    round((ss_qty*1.00)/NULLIF(COALESCE(ws_qty,0), 0), 2) AS ratio,
    ss_qty AS store_qty, ss_wc AS store_wholesale_cost,
    COALESCE(ws_qty,0) AS other_chan_qty
FROM ss
LEFT JOIN ws ON (ws_sold_year = ss_sold_year
                 AND ws_item_sk = ss_item_sk
                 AND ws_customer_sk = ss_customer_sk)
WHERE COALESCE(ws_qty,0) > 0 AND ss_sold_year = 2000
ORDER BY ss_sold_year, ss_item_sk, ss_customer_sk, ratio
LIMIT 100;

-- Query 37: Pattern 16 - GROUP BY with proper aggregation functions - Fixed
SELECT
    i_category,
    i_brand_id,
    ANY_VALUE(i_brand) AS sample_brand,  -- Use ANY_VALUE for non-grouped descriptive columns
    count(*) AS item_count,
    avg(i_current_price) AS avg_price,
    min(i_current_price) AS min_price,
    max(i_current_price) AS max_price
FROM item
GROUP BY i_category, i_brand_id  -- Both columns included in GROUP BY
ORDER BY i_category, avg_price DESC;

-- Query 38: Pattern 17 - Window function usage (no GROUP BY needed)
SELECT
    i_item_id,
    i_category,
    i_current_price,
    avg(i_current_price) OVER (PARTITION BY i_category) AS category_avg_price,
    i_current_price - avg(i_current_price) OVER (PARTITION BY i_category) AS price_diff_from_avg
FROM item
WHERE i_category IN ('Electronics', 'Clothing')
ORDER BY i_category, i_current_price DESC;

-- Query 39: Pattern 18 - Complex aggregation with proper grouping
SELECT
    s.s_state,
    d.d_year,
    d.d_qoy,
    count(DISTINCT ss.ss_customer_sk) AS unique_customers,
    count(*) AS total_transactions,
    sum(ss.ss_ext_sales_price) AS total_sales,
    avg(ss.ss_ext_sales_price) AS avg_transaction,
    ANY_VALUE(s.s_store_name) AS sample_store_name
FROM store s
INNER JOIN store_sales ss ON s.s_store_sk = ss.ss_store_sk
INNER JOIN date_dim d ON ss.ss_sold_date_sk = d.d_date_sk
WHERE d.d_year IN (2001, 2002)
GROUP BY s.s_state, d.d_year, d.d_qoy  -- All non-aggregate columns included
HAVING count(DISTINCT ss.ss_customer_sk) > 100
ORDER BY s.s_state, d.d_year, d.d_qoy;

-- Query 40: Pattern 19 - Subquery in SELECT with proper correlation
SELECT
    c.c_customer_id,
    c.c_first_name,
    c.c_last_name,
    (
        SELECT count(*)
        FROM store_sales ss
        INNER JOIN date_dim d ON ss.ss_sold_date_sk = d.d_date_sk
        WHERE ss.ss_customer_sk = c.c_customer_sk
        AND d.d_year = 2001
    ) AS purchases_2001,
    (
        SELECT COALESCE(sum(ss.ss_ext_sales_price), 0)
        FROM store_sales ss
        INNER JOIN date_dim d ON ss.ss_sold_date_sk = d.d_date_sk
        WHERE ss.ss_customer_sk = c.c_customer_sk
        AND d.d_year = 2001
    ) AS total_spent_2001
FROM customer c
WHERE c.c_preferred_cust_flag = 'Y'
ORDER BY total_spent_2001 DESC
LIMIT 100;



-- Query 41
SELECT DISTINCT
    i_category,
    i_class,
    i_brand_id
FROM item
WHERE i_current_price > 50
ORDER BY i_category, i_class;

-- Query 42: Pattern 22 - NULLIF and COALESCE for safe divisions and null handling
SELECT
    s_store_name,
    s_state,
    total_sales,
    total_returns,
    COALESCE(total_sales, 0) - COALESCE(total_returns, 0) AS net_sales,
    CASE WHEN NULLIF(total_sales, 0) IS NOT NULL
         THEN (total_returns / total_sales) * 100
         ELSE 0 END AS return_rate_pct
FROM (
    SELECT
        s_store_name,
        s_state,
        sum(ss_ext_sales_price) AS total_sales,
        sum(COALESCE(sr_return_amt, 0)) AS total_returns
    FROM store
    INNER JOIN store_sales ON s_store_sk = ss_store_sk
    LEFT JOIN store_returns ON ss_ticket_number = sr_ticket_number
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_year = 2001
    GROUP BY s_store_name, s_state
) store_metrics
ORDER BY return_rate_pct DESC
LIMIT 100;


-- Query 43:
SELECT
    c_customer_id,
    c_first_name,
    c_last_name,
    ss_sold_date_sk,
    ss_net_paid,
    -- Running total within date range (7 days before to current)
    SUM(ss_net_paid) OVER (
        PARTITION BY c_customer_sk
        ORDER BY ss_sold_date_sk
        RANGE BETWEEN 7 PRECEDING AND CURRENT ROW
    ) as rolling_7day_total,
    -- Average within monetary range
    AVG(ss_net_paid) OVER (
        PARTITION BY c_customer_sk
        ORDER BY ss_net_paid
        RANGE BETWEEN 100.00 PRECEDING AND 50.00 FOLLOWING
    ) as value_range_avg
FROM customer c
JOIN store_sales ss ON c.c_customer_sk = ss.ss_customer_sk
JOIN date_dim d ON ss.ss_sold_date_sk = d.d_date_sk
WHERE d_year = 2023
ORDER BY c_customer_sk, ss_sold_date_sk;

-- Query 44
WITH customer_daily_purchases AS (
    SELECT
        c_customer_id,
        d_date,
        sum(ss_ext_sales_price) as daily_spend
    FROM customer
    INNER JOIN store_sales ON c_customer_sk = ss_customer_sk
    INNER JOIN date_dim ON ss_sold_date_sk = d_date_sk
    WHERE d_year = 2001
    GROUP BY c_customer_id, d_date
)
SELECT *
FROM customer_daily_purchases
MATCH_RECOGNIZE (
    PARTITION BY c_customer_id
    ORDER BY d_date
    MEASURES
        FIRST(d_date) as trend_start,
        LAST(d_date) as trend_end,
        COUNT(*) as days_in_trend,
        AVG(daily_spend) as avg_daily_spend
    PATTERN (INCREASING_TREND{3,})
    DEFINE INCREASING_TREND AS daily_spend > LAG(daily_spend)
);


"""

#utility functions

In [58]:
# @title
def save_results(df,engine):
  df['engine']= engine
  df['time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  df['sf'] = SF
  df['cpu'] = core
  write_deltalake(result_path,df,mode="append",storage_options={"allow_unsafe_rename":"true"})
  return "done"

In [59]:
# @title
def execute_query(engine, sql_script, exclude_query=[]):
    results = []  # store rows as dicts
    sql_arr = [q.strip() for q in sql_script.split(";") if q.strip()]

    for index, query in enumerate(sql_arr, start=1):
        if index in exclude_query:
            print(f"Skipping Query {index}")
            continue

        print(f"Running Query {index}...")
        start = time.time()
        duration = 0

        try:
            engine_type = type(engine).__module__.split(".")[0]

            if engine_type == "polars":
                result_df = engine.execute(query).collect()
                print(result_df)
            elif engine_type == "chdb":
                result_df = engine.query(query,'dataframe')
                print(result_df)
            elif engine_type == "deltalake":
                result_df = engine.execute(query).read_all()
                print(result_df)
            else:
                engine.sql(query).show()

            duration = time.time() - start

        except Exception as er:
            print(f"Error in Query {index}: {er}")
            duration = 0

        print(f"Duration: {duration:.4f} seconds")
        results.append({'dur': duration, 'Query': index})

    # always return a valid DataFrame, even if empty
    df = pd.DataFrame(results, columns=['dur', 'Query'])
    return df

# ------------

# Run Queries

In [60]:
%%time
for tbl in tbl_list :
    duckdb.sql(f""" create or replace view {tbl}  as select * from delta_scan('{Table_Path}/CH{SF}/{tbl}')  """)
df = execute_query(duckdb,sql)
save_results(df,'duckdb')

Running Query 1...
┌───────────┬─────────────┬─────────────────┐
│ i_item_id │ i_item_desc │ i_current_price │
│  varchar  │   varchar   │  decimal(7,2)   │
├───────────┴─────────────┴─────────────────┤
│                  0 rows                   │
└───────────────────────────────────────────┘

Duration: 0.0924 seconds
Running Query 2...
┌──────────────────┬──────────────┬────────────────┐
│  c_customer_id   │ c_first_name │ ss_sales_price │
│     varchar      │   varchar    │  decimal(7,2)  │
├──────────────────┼──────────────┼────────────────┤
│ AAAAAAAAOHCFBAAA │ John         │          75.28 │
│ AAAAAAAAOHCFBAAA │ John         │          86.79 │
│ AAAAAAAAHJFOAAAA │ Helen        │          89.21 │
│ AAAAAAAAHJFOAAAA │ Helen        │          77.90 │
│ AAAAAAAAHJFOAAAA │ Helen        │          74.65 │
│ AAAAAAAAHJFOAAAA │ Helen        │          60.34 │
│ AAAAAAAAEIKPAAAA │ James        │          68.93 │
│ AAAAAAAAEIKPAAAA │ James        │          53.67 │
│ AAAAAAAAEIKPAAAA │ Jam

'done'

In [61]:
%%time
qb = QueryBuilder()
for tbl in tbl_list :
    qb.register(tbl, DeltaTable(f'{Table_Path}/CH{SF}/{tbl}'))
df = execute_query(qb,sql,[16])
save_results(df,'delta_rs')

Running Query 1...
arro3.core.Table
+-----------+-------------+------------------+
| i_item_id | i_item_desc | i_current_price  |
| Utf8      | Utf8        | Decimal128(7, 2) |
+-----------+-------------+------------------+
+-----------+-------------+------------------+
Duration: 0.0027 seconds
Running Query 2...
arro3.core.Table
+------------------+--------------+------------------+
| c_customer_id    | c_first_name | ss_sales_price   |
| Utf8             | Utf8         | Decimal128(7, 2) |
+------------------+--------------+------------------+
| AAAAAAAAOHCFBAAA | John         | 75.28            |
| AAAAAAAAOHCFBAAA | John         | 86.79            |
| AAAAAAAAHJFOAAAA | Helen        | 89.21            |
| AAAAAAAAHJFOAAAA | Helen        | 77.90            |
| AAAAAAAAHJFOAAAA | Helen        | 74.65            |
| AAAAAAAAHJFOAAAA | Helen        | 60.34            |
| AAAAAAAAEIKPAAAA | James        | 68.93            |
| AAAAAAAAEIKPAAAA | James        | 53.67            |
| AAAAAA

'done'

In [62]:
%%time
import chdb
conn = chdb.connect()
conn.query("CREATE DATABASE IF NOT EXISTS db_xxx  ; use db_xxx")
for tbl in tbl_list :
    conn.query(f" CREATE OR REPLACE view {tbl}  as select * from deltaLakeLocal('{Table_Path}/CH{SF}/{tbl}') ")
df = execute_query(conn,sql)
save_results(df,'chdb')
conn.close()

Running Query 1...
Empty DataFrame
Columns: [i_item_id, i_item_desc, i_current_price]
Index: []
Duration: 0.0288 seconds
Running Query 2...
           c_customer_id c_first_name ss_sales_price
0       AAAAAAAABAAAAAAA       Javier          66.66
1       AAAAAAAABAAAAAAA       Javier         119.78
2       AAAAAAAABAAAAAAA       Javier         175.24
3       AAAAAAAACAAAAAAA          Amy          74.88
4       AAAAAAAACAAAAAAA          Amy         152.10
...                  ...          ...            ...
796044  AAAAAAAABIPPAAAA        Linda          79.78
796045  AAAAAAAABIPPAAAA        Linda          65.00
796046  AAAAAAAABIPPAAAA        Linda          65.58
796047  AAAAAAAABIPPAAAA        Linda          50.49
796048  AAAAAAAABIPPAAAA        Linda         125.73

[796049 rows x 3 columns]
Duration: 0.5620 seconds
Running Query 3...
    i_category  item_count  avg_price
0        Music        1860  10.332020
1  Electronics        1812  10.117537
2     Children        1786   9.880017
3

In [63]:
%%time
from   pysail.spark import SparkConnectServer
from   pyspark.sql  import SparkSession
server = SparkConnectServer()
server.start()
_, port = server.listening_address
spark = SparkSession.builder.remote(f"sc://localhost:{port}").getOrCreate()
for tbl in tbl_list :
     spark.sql(f"""  CREATE or replace TABLE {tbl}   USING delta    LOCATION '{Table_Path}/CH{SF}/{tbl}'  """)
df = execute_query(spark,sql)
save_results(df,"lakesail")
spark.stop()

Running Query 1...
+---------+-----------+---------------+
|i_item_id|i_item_desc|i_current_price|
+---------+-----------+---------------+
+---------+-----------+---------------+

Duration: 0.0276 seconds
Running Query 2...
+----------------+------------+--------------+
|   c_customer_id|c_first_name|ss_sales_price|
+----------------+------------+--------------+
|AAAAAAAAOHCFBAAA|        John|         75.28|
|AAAAAAAAOHCFBAAA|        John|         86.79|
|AAAAAAAAHJFOAAAA|       Helen|         89.21|
|AAAAAAAAHJFOAAAA|       Helen|         77.90|
|AAAAAAAAHJFOAAAA|       Helen|         74.65|
|AAAAAAAAHJFOAAAA|       Helen|         60.34|
|AAAAAAAAEIKPAAAA|       James|         68.93|
|AAAAAAAAEIKPAAAA|       James|         53.67|
|AAAAAAAAEIKPAAAA|       James|         88.92|
|AAAAAAAAEIKPAAAA|       James|         60.74|
|AAAAAAAAEIKPAAAA|       James|        101.25|
|AAAAAAAAEIKPAAAA|       James|         95.47|
|AAAAAAAAEIKPAAAA|       James|        122.79|
|AAAAAAAAELBDAAAA|     R

In [64]:
%%time
import polars as pl
ctx = pl.SQLContext()
for tbl in tbl_list :
    ctx.register(tbl,pl.scan_delta(f'{Table_Path}/CH{SF}/{tbl}'))
df = execute_query(ctx,sql)
save_results(df,'polars')

/tmp/ipykernel_6530/3704272094.py:19: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  result_df = engine.execute(query).collect()


shape: (72_580, 1)
┌──────────────────┐
│ c_customer_id    │
│ ---              │
│ str              │
╞══════════════════╡
│ AAAAAAAABAAAAAAA │
│ AAAAAAAACAAAAAAA │
│ AAAAAAAADAAAAAAA │
│ AAAAAAAAEAAAAAAA │
│ AAAAAAAAFAAAAAAA │
│ …                │
│ AAAAAAAALJGIBAAA │
│ AAAAAAAANJGIBAAA │
│ AAAAAAAAOJGIBAAA │
│ AAAAAAAAPJGIBAAA │
│ AAAAAAAAAKGIBAAA │
└──────────────────┘
Duration: 0.3750 seconds
Running Query 5...
shape: (2, 2)
┌─────────┬─────────────┐
│ channel ┆ sales_count │
│ ---     ┆ ---         │
│ str     ┆ u32         │
╞═════════╪═════════════╡
│ Store   ┆ 2880404     │
│ Web     ┆ 719384      │
└─────────┴─────────────┘
Duration: 0.3933 seconds
Running Query 6...
Error in Query 6: unsupported function 'rank'
Duration: 0.0000 seconds
Running Query 7...
Error in Query 7: only equi-join constraints (on identifiers) are currently supported; found lhs=Identifier(Ident { value: "c_customer_sk", quote_style: None, span: Span(Location(9,27)..Location(9,40)) }), rhs=Identifier(Ide

'done'

In [ ]:
%%time
import daft
from   daft.session import Session
sess = Session()
for tbl in tbl_list :
    sess.create_temp_table(tbl, daft.read_deltalake(f'{Table_Path}/CH{SF}/{tbl}'))
df = execute_query(sess,sql,[24])
save_results(df,'daft')

countUInt64
107


Duration: 1.3929 seconds
Running Query 29...
Error in Query 29: Unsupported SQL: 'Subqueries are not supported'
Duration: 0.0000 seconds
Running Query 30...
Error in Query 30: Unsupported SQL: 'CASE with no else result'
Duration: 0.0000 seconds
Running Query 31...


🗡️ 🐟 Parquet Scan: 00:00 

🗡️ 🐟 Parquet Scan: 00:00 

🗡️ 🐟 Hash Join Build: 00:00 

No such comm: 0abd8eff326349928eb191bf2e0de5ea


🗡️ 🐟 Hash Join Build: 00:00 

🗡️ 🐟 Parquet Scan: 00:00 

🗡️ 🐟 Hash Join Build: 00:00 

No such comm: f9450b08ec22457494f18667100b38e6
No such comm: c6cac635cf7c4a4fa214c12970be076a
No such comm: 4591919dd20141c8a7e493f8e852e68b
No such comm: 9ead2a05e7d541d580310b2a7aba0e3a
No such comm: ee8922f7f29b44cdbb992b8bf2fecebd
No such comm: 4d845f1551c248a98eeaee81d056fca3
No such comm: 65febae02e2b4f4696b3ccfa2cb3b784
No such comm: e33c955a851e4cfbb2969450cb6e1477
No such comm: 10db19f986e64a209f788f6c4f13acb3
No such comm: 090743fc45614acbad3ced9ccbbd2fab
No such comm: a89a7506efb841bda5c2075fa44aa91c
No such comm: 02b5b552d5ac4deda2f318eecb952da1
No such comm: 08437d39a876452f896b34c6d895bab5
No such comm: e163e5f013af46c584c1e2c8aaabbb8b
No such comm: 5a10b298d74f4affbe568928051a3e04
No such comm: af72f1ad11ed495383c845d605d943db
No such comm: 773b9055939b4b89b77e4ae9495c587c
No such comm: b4f9087840854c5b97451199537713e4
No such comm: 2bb548c2a4464e3eb10807b23135a526
No such comm: beb177fe38584d129f4e5daa932b4c14
No such comm: 563af0c1e64e4d4c8e5571863d69f2a1
No such comm:

# Results

In [66]:
duckdb.sql(f"""
SELECT
    RANK() OVER (ORDER BY COUNT(distinct(query)) filter (WHERE dur > 0) DESC, avg(dur) filter (WHERE dur > 0) ASC) AS rank,
    engine as python_package,
    case when engine in('delta_rs','lakesail') then 'datafusion' when engine in('chdb') then 'clickhouse' else engine end as engine,
    case when engine in('daft','polars','lakesail') then 'delta_rs' else 'delta_kernel_rs' end as Delta_Reader,
    case when engine in('duckdb','chdb') then 'delta_rs(arrow-streaming)'  when engine in('polars') then 'delta_rs(arrow)' when engine in('delta_rs') then 'delta_rs' else 'native_parquet,delta_rs/metadata' end as Delta_Writer,
    COUNT(distinct(query)) filter (WHERE dur > 0) AS nbr_queries,
    floor(nbr_queries/44 *100) as percentage
FROM delta_scan('{result_path}')

GROUP BY ALL
ORDER BY nbr_queries   desc
""")

┌───────┬────────────────┬────────────┬─────────────────┬──────────────────────────────────┬─────────────┬────────────┐
│ rank  │ python_package │   engine   │  Delta_Reader   │           Delta_Writer           │ nbr_queries │ percentage │
│ int64 │    varchar     │  varchar   │     varchar     │             varchar              │    int64    │   double   │
├───────┼────────────────┼────────────┼─────────────────┼──────────────────────────────────┼─────────────┼────────────┤
│     1 │ duckdb         │ duckdb     │ delta_kernel_rs │ delta_rs(arrow-streaming)        │          43 │       97.0 │
│     2 │ delta_rs       │ datafusion │ delta_kernel_rs │ delta_rs                         │          39 │       88.0 │
│     3 │ lakesail       │ datafusion │ delta_rs        │ native_parquet,delta_rs/metadata │          34 │       77.0 │
│     4 │ chdb           │ clickhouse │ delta_kernel_rs │ delta_rs(arrow-streaming)        │          33 │       75.0 │
│     5 │ daft           │ daft       │ 